# COGS 108 - Data Checkpoint

# Names

- Baoni Li
- Huiyi He
- Jiayi Zhu
- Yuhan Zhou
- Yihuan Wang

<a id='research_question'></a>
# Research Question

What factors are associated with higher medical expenditure in adults in the United States? Specifically, are there relationships between medical costs and demographic factors such as age, working hours, and BMI, as well as health-related behaviors such as smoking behavior and overall lifestyle choices? If there are statistically significant relationships between these features and medical expenditure, are they positively or negatively associated?

# Dataset(s)

- Dataset Name: MEPS HC-224: 2020 Full Year Consolidated Data File
- Link to the dataset: https://meps.ahrq.gov/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-224
- Number of observations: 27805


- Dataset Name: MEPS HC-216: 2019 Full Year Consolidated Data File
- Link to the dataset: https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-216
- Number of observations: 28512


- Dataset Name: MEPS HC-209: 2018 Full Year Consolidated Data File
- Link to the dataset: https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-209
- Number of observations: 30461


- Dataset Name: MEPS HC-201: 2017 Full Year Consolidated Data File
- Link to the dataset: https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-201
- Number of observations: 31880


- Dataset Name: MEPS HC-192: 2016 Full Year Consolidated Data File	
- Link to the dataset: https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-192
- Number of observations: 34655


- Dataset Name: MEPS HC-181: 2015 Full Year Consolidated Data File	
- Link to the dataset: https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-181
- Number of observations: TBD


- Dataset Name: MEPS HC-171: 2014 Full Year Consolidated Data File	
- Link to the dataset: https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-171
- Number of observations: TBD


- Dataset Name: MEPS HC-163: 2013 Full Year Consolidated Data File
- Link to the dataset: https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-163
- Number of observations: TBD


- Dataset Name: MEPS HC-155: 2012 Full Year Consolidated Data File	
- Link to the dataset: https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-155
- Number of observations: TBD


- Dataset Name: MEPS HC-147: 2011 Full Year Consolidated Data File	
- Link to the dataset: https://meps.ahrq.gov/mepsweb/data_stats/download_data_files_detail.jsp?cboPufNumber=HC-147
- Number of observations: TBD



Above datasets are all collected from Medical Expenditure Panel Survey, which is a national survey conducted by the Agency for Healthcare Research and Quality (AHRQ) that collects data on healthcare utilization and expenditures. Each dataset contains information of the given variables for a year, combined, they represent data collected from year 2011-2020

# Setup

In [1]:
# pip install pyreadstat before import
import pandas as pd
import numpy as np
import os
import pyreadstat as pyreadstat

# Data Cleaning

Describe your data cleaning steps here.

In [2]:
# Read each dta file and store them in a list of dataframes
path = r'C:\Users\jojo1\OneDrive\Documents\UCSD\UCSD WI23\COGS 108\project'
files_dta = [file for file in os.listdir(path) if file.endswith('.dta')]

dfs = pd.DataFrame()
yrlst = np.arange(11,21,1)
for i in range(len(files_dta)):
    filepath = os.path.join(path, files_dta[i])
    df, meta = pyreadstat.read_dta(filepath)
    df['YEAR'] = np.repeat(yrlst[i], df.shape[0])
    df = df.rename(columns={col: col.replace(str(yrlst[i]), '_FINAL') for col in df.columns if str(yrlst[i]) in col})
    dfs = pd.concat([dfs, df])

In [3]:
dfs2 = dfs[['OFTSMK53', 'ADSMOK42', 'ADBMI42', 'BMINDX53', 'HOUR31', 
            'HOUR42', 'HOUR53', 'AGE_FINALX', 
            'RACEWX', 'RACEBX', 'RACEAX', 
            'REGION53', 'REGION42', 'REGION31', 'REGION_FINAL', 'YEAR']]

dfs2 = dfs2.reset_index()
dfs2 = dfs2.rename(columns={'index' : 'index_within_yr'})

In [4]:
######## clean smoking, merge how often smoke and adult smoke #########

In [5]:
dfs.OFTSMK53.value_counts()

 3.0     75632
-1.0     29073
 1.0      8966
 2.0      4211
-7.0       555
-8.0       210
-9.0        10
-15.0        1
Name: OFTSMK53, dtype: int64

In [6]:
dfs.ADSMOK42.value_counts()

 2.0    142164
-1.0    102851
 1.0     28228
-9.0      3333
Name: ADSMOK42, dtype: int64

In [7]:
###### drop if no BMI value ########

In [8]:
print(dfs2.ADBMI42.value_counts())
print(dfs2.BMINDX53.value_counts())

-1.0     24223
-15.0     1990
 25.8      696
 26.6      694
 25.1      578
         ...  
 54.6        1
 56.2        1
 52.7        1
 4.4         1
 10.8        1
Name: ADBMI42, Length: 542, dtype: int64
-1.0      61341
-9.0       4455
 26.6      3976
 25.8      3950
 25.1      3346
          ...  
 66.4         1
 76.8         1
 64.1         1
 66.7         1
 134.7        1
Name: BMINDX53, Length: 603, dtype: int64


In [9]:
######### clean region columns ############

# make sure no rows in data has all four region columns' value unuseable
def drop_null_regions(df):
    slice1 = df[df['REGION_FINAL'] == -1].copy()
    slice2 = slice1[slice1['REGION53'] == -1].copy()
    slice3 = slice2[slice2['REGION42'] == -1].copy()
    slice4 = slice3[slice3['REGION31'] == -1].copy()

    if slice4.shape[0] == 0:
        None
    else:
        df.drop(index=[slice4.index])

# merging regions
def merge_region(row):
    if row['REGION_FINAL'] == -1:
        if row['REGION53'] == -1:
            if row['REGION42'] == -1:
                return f"{row['REGION31']}"
            else:
                return f"{row['REGION42']}"
        else:
            return f"{row['REGION53']}"
    else:
        return f"{row['REGION_FINAL']}"

    
dfs2['REGION_Merged'] = dfs2.apply(merge_region, axis=1)
dfs2.drop(columns=['REGION_FINAL', 'REGION53', 'REGION42', 'REGION31'])

,index_within_yr,OFTSMK53,ADSMOK42,ADBMI42,BMINDX53,HOUR31,HOUR42,HOUR53,AGE_FINALX,RACEWX,RACEBX,RACEAX,YEAR,REGION_Merged
0,0,NaN,2.0,NaN,39.1,48.0,-2.0,-2.0,30.0,1.0,3.0,3.0,11,1.0
1,1,NaN,2.0,NaN,25.8,-1.0,-1.0,-1.0,27.0,1.0,3.0,3.0,11,1.0
2,2,NaN,-1.0,NaN,-1.0,-1.0,-1.0,-1.0,6.0,1.0,3.0,3.0,11,1.0
3,3,NaN,-1.0,NaN,-1.0,-1.0,-1.0,-1.0,5.0,1.0,3.0,3.0,11,1.0
4,4,NaN,2.0,NaN,20.2,40.0,-2.0,-2.0,53.0,3.0,3.0,2.0,11,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334837,27800,2.0,NaN,26.6,NaN,40.0,-2.0,40.0,43.0,3.0,1.0,3.0,20,3.0
334838,27801,3.0,NaN,24.0,NaN,30.0,-2.0,-2.0,34.0,3.0,1.0,3.0,20,3.0
334839,27802,-1.0,NaN,-1.0,NaN,-1.0,-1.0,-1.0,14.0,3.0,1.0,3.0,20,3.0
334840,27803,-1.0,NaN,-1.0,NaN,-1.0,-1.0,-1.0,12.0,3.0,1.0,3.0,20,3.0


In [10]:
dfs2.REGION_Merged.value_counts()

3.0    127566
4.0     90339
2.0     64846
1.0     52091
Name: REGION_Merged, dtype: int64

In [ ]:
dfs2.to_stata('mydata.dta')
df = pd.read_stata('mydata.dta')
df = df.fillna(0)
df = df.drop(columns=['REGION_FINAL', 'REGION53', 'REGION42', 'REGION31'])
# drop BMI
bmi_df = df.get(['ADBMI42', 'BMINDX53'])
df = df[(bmi_df > 0).any(axis=1)]
# df = df[bmi_df.notnull()].copy()
df['BMI_Merged'] = bmi_df.apply(lambda frame: (frame['ADBMI42'] 
                                + frame['BMINDX53'])/2 if (frame['ADBMI42'] 
                                > 0 and frame['BMINDX53'] > 0) else max(frame['ADBMI42'], frame['BMINDX53']), axis=1)
df.get(['BMI_Merged'])
df = df.drop(columns = ['ADBMI42', 'BMINDX53'])
# drop Smoking
smoke_df = df.get(['OFTSMK53', 'ADSMOK42'])
df = df[(smoke_df > 0).any(axis=1)]
df['SMOKE_Merged'] = smoke_df.apply(lambda frame: (frame['OFTSMK53'] 
                                + frame['ADSMOK42'])/2 if (frame['OFTSMK53'] 
                                > 0 and frame['ADSMOK42'] > 0) else max(frame['OFTSMK53'], frame['ADSMOK42']), axis=1)
df.get(['SMOKE_Merged'])
df = df.drop(columns = ['OFTSMK53', 'ADSMOK42'])
df = df[df['AGE_FINALX'] >= 21.0]
# df.drop(columns = ['index', 'index_within_yr'])
df
hour=df[['HOUR31','HOUR42','HOUR53']]
hour = hour.applymap(lambda x: np.nan if x <= 0 else x)
hour = hour.dropna(how='all')
hour['num_notna']=hour.notna().sum(axis=1)
hour=hour.fillna(0)
hour['sum']=hour['HOUR31']+hour['HOUR42']+hour['HOUR53']
hour['HOUR_Merged']=hour['sum']/hour['num_notna']
hour=hour['HOUR_Merged']
final=df.merge(hour,how='right',right_index=True,left_index=True)
final=final.drop(columns=['HOUR31','HOUR42','HOUR53','index','index_within_yr']).reset_index()
final=final.drop(columns='index')
final